aws rpc wrapper?


In [ ]:
var importer = require('../Core');
var assert = require('assert');
var getResult = importer.import('rpc result');

function handler(event, context, callback) {
    var body = event || {};
    try {
        if (event.body || event.queryStringParameters) {
            body = Object.assign(event.body || {}, event.queryStringParameters || {});
        }

        assert(body.function && body.function !== '', 'something is wrong with your search ' + JSON.stringify(body));
        assert(importer.interpret(body['function']).id, 'something is terrible wrong with function lookup ' + importer.interpret(body['function']))
    }
    catch(e) {
        callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        })
        return;
    }
    
    // TODO: add Eloqua Notify service entry point for retrieving temporary data?
    // TODO: parse action and call from notify service or call with posted data?
    // TODO: add an entry point for Zuora subscription callout to update single records in eloqua?
    return Promise.resolve([])
        .then(() => getResult({
            command: body['function'],
            result: importer.interpret(body['function']),
            body: body,
            circles: ['Public']
        }))
        .then(r => callback(null, {
            'statusCode': 200,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify(r, null, 4)
        }))
        // TODO: object assign error?
        .catch(e => callback(e, {
            'statusCode': 500,
            'headers': { 
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*'
            },
            'body': JSON.stringify({'Error': e.message})
        }));
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}
module.exports.handler = handler;
